<a href="https://colab.research.google.com/github/marcchoueiry/livedrop-marc-choueiry/blob/main/notebooks/llm-deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch sentence-transformers torch sentence-transformers faiss-cpu flask pyngrok

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
            

KeyboardInterrupt: 

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import faiss
from flask import Flask, request, jsonify
from pyngrok import ngrok
import json

In [ ]:
KNOWLEDGE_BASE = [
    {"id": "doc1", "title": "Shoplite User Registration Process", "content": "To create a Shoplite account, users visit the registration page and provide their email address, password, and basic profile information. Email verification is required within 24 hours of registration. Users can choose between buyer accounts (free) or seller accounts (requires business verification).

Once registered, buyers can immediately browse products, add items to the cart, and explore features like wishlists or saved items. Buyers have access to order tracking, reviews, and applying promo codes during checkout. Seller accounts, on the other hand, allow users to manage inventory, add products with descriptions and images, track sales, handle returns, and configure shipping options. Business verification is required for sellers, which may include submitting a company name, tax ID, and contact details.

For account management, users can edit personal and account information. Changing the password requires entering the old password, then typing the new one twice. If the old password is correct and the new ones match, the password is updated, and a confirmation message “Password changed successfully” is shown. Personal info like name and date of birth can only be edited twice, and after that, users must wait one month before making further changes.

Seller accounts have additional features. They can update contact info such as email, phone, and address anytime. They also need to complete business information to gain trust from buyers and enable smooth order processing. The dashboard allows sellers to track stock, view pending orders, and manage promotions or discounts.

Shoplite keeps everything synced across devices. Users can log in from a phone, tablet, or computer, and see their cart, orders, or inventory without losing data. Notifications keep users informed about important updates, like account changes or verification status.

Overall, the registration process is designed to be simple, secure, and functional. It provides all the necessary features depending on whether the user is a buyer or a seller, while keeping rules clear for passwords, personal info edits, and business verification.

"},
    {"id": "doc2", "title": "Shoplite Shopping Cart Features", "content": "The Shoplite shopping cart allows users to add multiple items from different sellers, apply promotional codes, and save items for later purchase. Cart contents are preserved for logged-in users across sessions, so users can log in from another device and still find everything they added.

Users can update quantities of items, remove products they no longer want, or move them to a “save for later” list if they’re not ready to buy immediately. For example, if someone adds a phone and a case, they might purchase the phone now but save the case for later. The cart automatically splits items by seller at checkout since each seller fulfills their own orders. Users can also choose different shipping options per seller if available.

Promotional codes can be applied either in the cart or during checkout. If a code is invalid, expired, or doesn’t meet minimum requirements, the system notifies the user. Prices automatically update if a product changes price while in the cart, and out-of-stock items are flagged with alerts so users don’t try to purchase unavailable items.

The cart shows a summary with subtotal, taxes, shipping, discounts, and the total amount, making it clear how much the user will pay. Shoplite also suggests related or complementary products under the cart, like accessories for electronics or matching items for clothing, giving users a chance to find what they might need.

Cross-device syncing ensures that items added on a phone appear on a computer or tablet. Users also get notifications if a saved item is running low in stock or if a promotion is ending soon. Sellers cannot change what’s in the buyer’s cart, but the system automatically updates availability and pricing if needed.

Overall, the Shoplite shopping cart is designed to be flexible, informative, and convenient, giving users full control over what they want to buy while making sure everything stays accurate and up to date across devices and sessions."},
    {"id": "doc3", "title": "Shoplite Checkout and Payment Flow", "content": "When a user finishes adding items to the cart, they go to the checkout page to finalize the order. First, they confirm the shipping address and select a delivery method. Users can save multiple addresses in their profile and pick the one they want for each order. Then they choose a payment option, like credit or debit card, PayPal, cash on delivery if the seller allows it, or Shoplite Wallet credits. The system applies any discounts or promotional codes, and shows the final total before confirming the payment.

If the payment succeeds, the order is confirmed and the seller is notified to prepare the items. If the payment fails, the user can try again, choose a different method, or contact support. For cash on delivery, the order is marked pending until the delivery is completed. Users are also shown an estimated delivery date based on the shipping option they selected.

The checkout page is connected to the cart, so if any item went out of stock while the user was still shopping, the system will alert them before placing the order. Users can also see a summary including subtotal, shipping, taxes, discounts, and total. If there are errors, like an invalid promo code or insufficient wallet balance, the system notifies the user with clear messages. Additionally, Shoplite suggests add-ons or complementary items during checkout, like a phone case or charger if a phone is being purchased.

Finally, Shoplite keeps a copy of the invoice in the user’s account, so they can view it anytime. Users can also choose gift wrapping or special instructions if the seller offers it. The checkout process is designed to be simple but flexible, letting users review and change anything before confirming the payment.

"},
    {"id": "doc4", "title": "Shoplite Order Tracking and Delivery", "content": "After an order is placed, the user receives an order ID and can track the status from their account. The order status updates automatically, showing stages like “Processing,” “Shipped,” “Out for Delivery,” and “Delivered.” Sellers update the status from their dashboard, and Shoplite syncs with delivery partners to provide live tracking. Users get notifications by email or in the app whenever the status changes.

Shoplite supports multiple shipping addresses, so users can send items to different locations and track each order separately. If a shipment is delayed, the system sends an alert with an explanation and the new estimated delivery date. For deliveries, some items may require a signature or PIN to confirm receipt. If a user is unavailable, the delivery partner may attempt redelivery or leave the item at a secure pickup location.

Partial shipments are supported too. For example, if a user orders multiple products from the same seller, some items may be shipped immediately while others take longer. Users can see which items are included in each shipment. Shoplite also allows users to contact support from the tracking page if there is a problem, like a missing package or damaged product.

Notifications are designed to keep users informed at every step. Users can opt in for push notifications on mobile or SMS alerts. The system also records delivery history in the account so users can check past orders. Shoplite tries to make tracking simple but reliable, giving users control and visibility until their order reaches them.

"},
    {"id": "doc5", "title": "Shoplite Return and Refund Policies", "content": "Shoplite allows users to request a return within 14 days from the delivery date. The item must be unused, undamaged, and in its original packaging. To start a return, users go to “My Orders,” select the item, and choose “Request Return.” The seller receives the request and must approve it. Once approved, a return label is issued so the user can send the item back.

After the item is returned and inspected, the refund is processed. Users can get their money back through the original payment method or Shoplite Wallet credits. Refund times vary: wallet refunds are usually instant, while card refunds can take 5–7 business days. Some products, like perishable goods, digital downloads, or personalized items, are not eligible for returns.

If a return is rejected, for example because the item is damaged or used, it is sent back to the user. Shoplite may provide step-by-step instructions to make the process easier, including how to package the item, print the label, and schedule a pickup. Users are also notified at each step: when the return is approved, when the item is received, and when the refund is issued.

Sellers can have additional return rules, like specifying certain courier partners. Users should check the seller’s policy alongside Shoplite’s general rules. Overall, the system is designed to protect buyers while giving sellers clear steps to handle returns efficiently."},
    {"id": "doc6", "title": "Shoplite Product Reviews and Ratings", "content": "After receiving their order, buyers on Shoplite can leave a review and rating for the product. Ratings range from 1 to 5 stars, and buyers can also write comments describing their experience and upload photos of the product. Verified Purchase badges ensure that only actual buyers can leave reviews, which helps other users trust the feedback.

Reviews are important for both buyers and sellers. Buyers can filter reviews by rating, date, or whether images are included. Sellers can respond to reviews, thanking buyers or clarifying issues. Negative reviews remain visible to maintain transparency, but spam or abusive comments can be reported and removed after Shoplite review.

Each review contributes to the product’s overall rating, which is displayed prominently on the product page. High-rated products gain more visibility in search results, encouraging more sales, while consistently low ratings can alert sellers to potential quality issues. Shoplite also analyzes reviews to detect patterns like frequent complaints about a product defect or shipping delays, helping the platform improve the overall experience.

Buyers can edit their reviews within a limited time frame, for example to add more details or correct mistakes. Shoplite stores review history, so users can see past feedback and maintain accountability. Images in reviews give other buyers a realistic view of the product’s condition. Sellers often use reviews to improve products, adjust descriptions, or fix recurring issues.

Overall, the review system balances transparency with usability. It gives buyers the confidence to make informed purchases and provides sellers with actionable insights. Honest feedback is encouraged, highlighting both positives and negatives, which strengthens trust in the Shoplite platform and ensures a better shopping experience for everyone."},
    {"id": "doc7", "title": "Shoplite Seller Account Setup", "content": "To become a seller on Shoplite, users must create a seller account. This involves submitting business information like company name, tax ID, address, and contact details. Shoplite reviews all submissions before approving the account to make sure everything is valid. Once approved, sellers can access their dashboard to start listing products, managing orders, and monitoring sales.

Sellers are encouraged to complete their profile fully. Adding a logo, a detailed shop description, and branding images helps buyers trust the shop. They can configure shipping methods, return policies, and promotional campaigns to attract customers. The dashboard provides tools to answer buyer questions, manage pending orders, and handle returns efficiently.

Shoplite sets performance standards for sellers. Orders must be shipped on time, inventory levels kept accurate, and customer service responsive. Repeated failures can trigger warnings or account suspension. Analytics in the dashboard show trends, top-selling items, and revenue reports, helping sellers plan restocking, promotions, or expanding their inventory strategically.

Sellers also need to understand fees and commissions. Shoplite deducts a percentage of each sale, and additional fees may apply for optional services like promoted listings. Transparent reporting allows sellers to know exactly how much they earn after fees, which helps with pricing decisions and business planning.

Overall, the seller account setup is designed to give new sellers the tools, guidance, and visibility they need to succeed while ensuring a smooth experience for buyers. Completing all steps carefully sets up a strong foundation for growing their business on Shoplite."},
    {"id": "doc8", "title": "Shoplite Inventory Management for Sellers", "content": "Managing inventory is a core responsibility for sellers on Shoplite, and the platform provides several tools to make it easier. Each product listing requires details such as name, description, category, price, and available stock. Sellers can also manage variations like size, color, or style, with each variation having its own stock count. Keeping inventory accurate is essential to avoid overselling, prevent cancellations, and maintain buyer trust.

When an order is placed, Shoplite automatically updates the stock, and sellers receive low-stock alerts when quantities reach a predefined threshold. This allows them to restock items before they sell out. If a buyer has an item in their cart and it goes out of stock before checkout, the system immediately notifies them, preventing disappointment or abandoned orders.

Shoplite supports inventory management across multiple locations or warehouses, which is useful for sellers handling large volumes. Bulk upload options make it simple to add hundreds of products at once. Analytics tools provide insights on top-selling products, helping sellers decide which items to prioritize for restocking or promotion. Pre-orders and backorders are supported, enabling sellers to accept orders even if stock is temporarily unavailable.

Sellers can also adjust prices and stock levels at any time, which is particularly useful for seasonal products, flash sales, or limited editions. Notifications alert sellers to items that require attention, such as perishable goods nearing expiration. By maintaining accurate and organized inventory, sellers ensure that buyers have a smooth experience, avoid stock-related issues, and build a strong reputation on Shoplite. Overall, the inventory management system combines automation, analytics, and flexibility to help sellers operate efficiently and grow their business."},
    {"id": "doc9", "title": "Shoplite Commission and Fee Structure", "content": "Shoplite charges sellers a commission for each completed sale, which is typically a percentage of the product price. The exact percentage varies by product category and can range from 5% to 15%. Additional fees may apply for optional services like promoted listings, premium account features, or certain payment methods. Sellers can view all commissions and fees in their dashboard, providing transparency and helping them understand their net earnings.

Refunds, returns, and chargebacks automatically deduct the corresponding amounts from the seller’s balance. Shoplite provides detailed transaction reports for every sale, showing commissions, deductions, and expected payouts. Payouts are usually processed weekly, either to the seller’s linked bank account or Shoplite Wallet, ensuring sellers can manage their cash flow efficiently.

Understanding the fee structure is essential for pricing products correctly. Sellers need to account for commissions and any additional charges when setting prices to maintain profitability. Transparent reporting helps sellers plan promotions, restock inventory, and evaluate their business performance. By clearly showing how fees are calculated and applied, Shoplite fosters trust between the platform and sellers. This transparency also helps prevent disputes and encourages sellers to focus on growing their sales rather than worrying about hidden costs.

Ultimately, Shoplite’s commission and fee system is designed to balance the platform’s revenue needs with seller profitability. It encourages sellers to use the platform effectively, while providing clear information about costs so they can make informed decisions about pricing, marketing, and inventory management.

"},
    {"id": "doc10", "title": "Shoplite Customer Support Procedures", "content": "Shoplite provides comprehensive customer support to help both buyers and sellers resolve issues efficiently. Buyers can report problems with orders, payments, delivery delays, or damaged products, while sellers can request help with account verification, payout issues, or order disputes. Each support request generates a ticket, which allows Shoplite and the user to track progress and follow up if needed.

Support is available through multiple channels: live chat, email, and in-app messaging. Urgent issues, such as payment failures or missing orders, are prioritized to ensure timely resolution. For disputes between buyers and sellers, Shoplite acts as a mediator by reviewing transaction history, messages, and system logs, helping reach fair outcomes without unnecessary delays.

The Help Center contains detailed guides, FAQs, and tutorials for common problems. This includes instructions for password recovery, initiating refunds, tracking shipments, and managing account settings. Sellers receive guidance on platform policies, performance expectations, and best practices to prevent future disputes.

Response times vary depending on issue complexity, but Shoplite aims to resolve critical problems quickly. Each ticket keeps a full history of communications, so users can check updates or review past interactions. Overall, Shoplite’s support system ensures that issues are handled efficiently, fairly, and transparently. It provides confidence to buyers and sellers alike, giving them clear pathways to resolve problems and maintain trust in the platform.

"},
    {"id": "doc11", "title": "Shoplite Mobile App Features", "content": "The Shoplite mobile app is designed to provide a seamless shopping experience for buyers and flexible management tools for sellers. Available for iOS and Android, the app allows buyers to browse products, filter search results, add items to the cart, apply promotional codes, and complete checkout. Saved carts and wishlists synchronize across devices, so users can continue shopping seamlessly whether they switch from phone to tablet.

Push notifications keep buyers informed about order updates, delivery status, new promotions, and limited-time deals. The app supports biometric login like fingerprint or face recognition, making access fast and secure. Some pages can be accessed offline to view previously loaded products, but completing purchases requires an internet connection.

Sellers can also benefit from the mobile app. They can monitor sales, manage inventory, respond to buyer questions, receive low-stock alerts, and update product listings directly from their phone. Real-time analytics allow sellers to track revenue trends, pending tasks, and performance metrics, enabling them to stay on top of their business while on the go.

The app emphasizes usability and speed. Buyers can quickly filter by category, price, or rating, and product pages load efficiently. Sellers have a dashboard view for their products, orders, and analytics. Frequent updates improve performance, security, and user experience, ensuring that Shoplite’s mobile app remains a reliable tool for shopping and business management anytime, anywhere.

"},
    {"id": "doc12", "title": "Shoplite API Documentation for Developers", "content": "Shoplite provides a RESTful API that allows developers to integrate Shoplite functionality into external applications, dashboards, or analytics platforms. The API supports key operations such as searching for products, creating and updating orders, managing user accounts, and modifying inventory. Developers need an API key for authentication, and endpoints are subject to rate limits to prevent abuse.

API responses are delivered in JSON format. Each endpoint comes with clear documentation, including request examples, expected responses, error codes, and usage notes. Common error codes include 401 for unauthorized access, 404 for resource not found, and 429 for too many requests. Developers can handle these codes to build robust error management systems in their applications.

Developers can use the API for a variety of purposes. For example, embedding Shoplite search on a website, tracking inventory across multiple locations, or creating custom analytics dashboards for sellers. The API also provides endpoints for retrieving order history, transaction details, and performance metrics, which can be used to automate reporting, notifications, and inventory updates.

Security is emphasized throughout the API. Developers must keep their API keys secure, use encryption when transmitting data, and limit access to authorized functions. Following these guidelines ensures that external applications integrate reliably while maintaining the privacy and security of Shoplite users.

"},
    {"id": "doc13", "title": "Shoplite Security and Privacy Policies", "content": "Shoplite takes the security and privacy of users very seriously. All sensitive data is transmitted using SSL encryption, and passwords are securely hashed to prevent unauthorized access. Users can enable two-factor authentication to add an extra layer of protection. Suspicious login attempts trigger alerts, and accounts may be temporarily locked until the owner confirms identity.

Personal information is only shared with sellers when necessary to fulfill orders. Shoplite does not sell or share user data for marketing purposes. Users can manage preferences for marketing communications, and they can request data export or deletion in compliance with GDPR and other privacy regulations.

Regular security audits are performed to identify vulnerabilities and strengthen defenses. In the event of a data breach, affected users are notified promptly. Internal access to sensitive data is strictly controlled, and staff members are trained to handle information responsibly. These measures help maintain a safe environment for both buyers and sellers, fostering trust in Shoplite’s platform while preventing misuse or fraud.

"},
    {"id": "doc14", "title": "Shoplite Promotional Codes and Discounts", "content": "Shoplite allows buyers to use promotional codes to get discounts, free shipping, or special offers. These codes can be Shoplite-wide or specific to a seller, and each code may include rules such as expiry dates, minimum purchase amounts, or applicable products.

During checkout, discounts are automatically applied and reflected in the order total. If a code is invalid, expired, or not applicable, the system notifies the buyer immediately. Some codes are limited to first-time users, while others can be reused according to seller or Shoplite rules. Shoplite tracks code usage to prevent abuse and ensure fairness.

Sellers can create their own promotions from the dashboard, choosing the type of discount, applicable products, duration, and any restrictions. Promotional codes can boost visibility, attract new buyers, and encourage repeat purchases. Shoplite also provides reporting tools to monitor code usage and performance, allowing sellers to optimize their campaigns.

By providing flexible options for promotions, Shoplite helps sellers market effectively while giving buyers extra incentives to shop. Clear rules and notifications ensure transparency and reduce confusion, creating a smooth and satisfying shopping experience for everyone."},
    {"id": "doc15", "title": "Shoplite Account Security and Recovery", "content": "Shoplite provides multiple tools to protect user accounts and ensure safe access. Two-factor authentication can be enabled to add a layer of security. Users are notified about logins from new devices or unusual activity, helping them quickly detect unauthorized access.

If a password is compromised, users can reset it via the “Forgot Password” option, which sends a secure link to the registered email. Additional verification, such as past order details or ID confirmation, may be required to confirm account ownership. Users can review login history and manage active sessions, logging out remotely if needed.

In cases of suspected fraud, Shoplite may temporarily freeze accounts while supporting users through the recovery process. Strong, unique passwords are recommended, and users should update them periodically. Support staff assist with identity verification and account recovery, ensuring users regain access safely.

Shoplite’s approach balances security with usability, making it easy for legitimate users to manage accounts while protecting the platform from unauthorized access and fraud. The system gives users confidence that their personal data, order history, and financial information remain secure while maintaining convenient access to their accounts."},
]

# RAG Demo Notebook

This notebook will implement the RAG system step by step.

In [ ]:
print("Notebook loaded correctly!")